## Python wrapper for SWMM
Test case for Gottesacker model by Zhao Chen.
***
**Goals:**
<br>- Writes SWMM project.inp files using a template (template.inp) - searches for placeholder strings in the template and replaces them with data from a pandas dataframe. Dataframes can be created manually, or loaded from .csv files.
<br>- Run SWMM using the specified project.inp file
<br>- Import results from project.rpt file as pandas dataframes
***
**Files needed:**
<br>- template.inp
<br>- swmm5.exe
<br>- swmm5.dll
<br>
**Packages needed:**
<br>- pandas
<br>-

In [113]:
import pandas as pd
import numpy as np

In [274]:
#File structure and imports:

template_filename = 'template.inp'                                          #filename for template .inp file to import
input_filename =    'test.inp'                                              #filename to write new .inp file to
data_filename =     'junctions.csv'                                         #filename with data to insert into the .inp file
placeholder =       'junctions'                                             #placeholder string to replace with data

template =   pd.read_csv(template_filename, header=None, skip_blank_lines=False) #import template .inp file to dataframe
temp =       template.copy()                                                     #make a copy of the template, so that modifications won't change the original
data =       pd.read_csv(data_filename,     header=1)                            #import data to dataframe that has separate columns


In [275]:
data.head()

,Name,InvertElev,MaxDepth,InitDepth,SurDepth,Aponded
0,1001,1900.0000,0,0,200,0
1,1002,1731.3889,0,0,200,0
2,1003,1624.0892,0,0,200,0
3,1004,1516.7895,0,0,200,0
4,1005,1409.4897,0,0,200,0


In [276]:
#Get row index and location of placeholder:
ind = temp.index[temp[0]=='junctions']                      #df.index gets the index, df[0] looks in column 0 (returns an index object not just the name)
loc = temp.index.get_loc(ind[0])                            #get the integer position of the index specified (need to select first item (index name) in index object)

#Convert data to formatted line strings:

#Create format string based on template file:
fmt_line = temp[0].loc[loc-1]                            #get string to model formatting on (the row of dashes in the template file)
cols = fmt_line.split()                                  #split string into chunks separated by whitespace
l = [len(item) for item in cols]                         #get length of each column
form = ''                                                #create empty string to fill
form = [form + '{d['+ str(i) + ']:<' + str(l[i]+1) + '}' for i in range(len(l))]  #concatenate formatting info and return a list of format strings (one per column)
form = ''.join(form)                                     #join format strings into one for entire row

#Insert values into format string
data_strings = pd.DataFrame(columns=[0])            #create empty dataframe to fill
for ind in data.index:                              #loop over lines of data
    dl = data.loc[ind].tolist()                     #get line of data to be formatted
    dl[0] = int(dl[0])                              #make sure name column is an integer not a float (WHY is this not automatic?)
    line = form.format(d=dl)                        #insert each item in list into format string
    data_strings.loc[ind] = line                    #insert line string into new dataframe


In [277]:
#Replace placeholder with new data:

#Split original df into two, one for everything before the placeholder, one for everything after:
dfA = temp[temp.index < loc]                                    #create df for everything above placeholder
dfB = temp[temp.index > loc]                                    #create df for everything below placeholder

#Append the three dfs to each other (part above, part to insert, part below):
temp = dfA.append( data_strings, ignore_index=True)             #append additional part to top part
temp = temp.append(dfB,       ignore_index=True)                #append bottom part to new df

print(temp.iloc[loc-2:loc+len(data_strings)])                   #print updated section


                                                     0
177  ;;Name           InvertElev MaxDepth   InitDep...
178  ;;-------------- ---------- ---------- -------...
179  1001             1900.0     0.0        0.0    ...
180  1002             1731.3889  0.0        0.0    ...
181  1003             1624.0892  0.0        0.0    ...
182  1004             1516.7895  0.0        0.0    ...
183  1005             1409.4897  0.0        0.0    ...
184  1006             1900.0     0.0        0.0    ...
185  1007             1710.6706  0.0        0.0    ...
186  1008             1587.5349  0.0        0.0    ...
187  1009             1464.3991  0.0        0.0    ...
188  1010             1341.2634  0.0        0.0    ...
189  1011             1700.0     0.0        0.0    ...
190  1012             1551.6158  0.0        0.0    ...
191  1013             1441.628   0.0        0.0    ...
192  1014             1331.6403  0.0        0.0    ...
193  1015             1221.6526  0.0        0.0    ...
194  1016 

In [278]:
temp.to_csv(input_filename, header=False, index=False)                     #write dataframe to .inp text file with specified name